## Imports

In [2]:
import joblib
import pandas as pd
import datetime
import requests
import censusgeocode as cg
import ipywidgets as widgets
from ipywidgets import HBox, VBox

In [89]:
df_bus = pd.read_csv('../data/bus_station_data_full.csv')

In [90]:
df_bus['census_index'] = df_bus['census_index'].apply(str)
df_bus['census_index'] = df_bus['census_index'].apply(lambda x: x.zfill(7))

In [92]:
df_bus.head()

,Unnamed: 0,Latitude,Longitude,Stop_Name,Routes_Available,Stop_ID,census_block,census_blockgroup,census_tract,census_index
0,0,38.993831,-77.029731,SSTC ML COLLECTION POINT I/B,"70, 79, F4, J4, Q1, Q2, Q2v1, Q2v2, Q4, Y2, Y7...",0,4003,4,702500,7025004
1,1,39.077106,-76.953772,Z11 EXPRESS ANOUNCEMENT,"Z11, Z11v1",0,1010,1,701414,7014141
2,2,38.920629,-77.073797,BENTON ST NW + TUNLAW RD NW,"D1, D2",1001773,4005,4,300,0003004
3,3,38.920760,-77.021924,GEORGIA AVE NW + BARRY PL NW,70,1001774,2007,2,3400,0034002
4,4,38.920636,-77.076667,BENTON ST NW + 39TH ST NW,"D1, D2",1001775,4009,4,300,0003004


## Load Dan's model from file

In [14]:
filename = 'RandomForestClassifier.sav'
loaded_model = joblib.load(filename)

/anaconda3/lib/python3.6/site-packages/sklearn/base.py:253: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 0.21.2 when using version 0.20.3. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/anaconda3/lib/python3.6/site-packages/sklearn/base.py:253: UserWarning: Trying to unpickle estimator RandomForestClassifier from version 0.21.2 when using version 0.20.3. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


## Define function to geocode address input and return census block info

In [15]:
def addressLookup(address_input):
    address = cg.onelineaddress(address_input)

    addressLookup.block_group = address[0]['geographies']['2010 Census Blocks'][0]['BLKGRP']
    addressLookup.tract = address[0]['geographies']['2010 Census Blocks'][0]['TRACT']
    addressLookup.lat = address[0]['coordinates']['y']
    addressLookup.lon = address[0]['coordinates']['x']

## Define function to use geocode of address input to get weather info

In [16]:
def darkSkyAPICall(lat,lon,date_time):

    base_url = 'https://api.darksky.net/forecast/'
    api_key = 'c9274e7c52c1a5b7e99be6f22db98855'
    exclude = 'minutely, hourly, daily, flags'
    params = {'exclude': exclude}

    lat_address = lat
    lon_address = lon

    query = "/{},{}".format(lat_address,lon_address,date_time)

    url = base_url + api_key + query

    try:
        response = requests.get(url, params=params)
    except ConnectionError:
        pass

    try:
        response_json = response.json()
    except:
        response_json = {}

    darkSkyAPICall.summary = response_json['currently']['summary']
    darkSkyAPICall.precip_intensity = response_json['currently']['precipIntensity']
    darkSkyAPICall.precip_probability = response_json['currently']['precipProbability']
    darkSkyAPICall.temp = response_json['currently']['temperature']
    darkSkyAPICall.dewpoint = response_json['currently']['dewPoint']
    darkSkyAPICall.humidity = response_json['currently']['humidity']
    darkSkyAPICall.pressure = response_json['currently']['pressure']
    darkSkyAPICall.wind_speed = response_json['currently']['windSpeed']
    darkSkyAPICall.wind_gust = response_json['currently']['windGust']
    darkSkyAPICall.wind_bearing = response_json['currently']['windBearing']
    darkSkyAPICall.cloud_cover = response_json['currently']['cloudCover']
    darkSkyAPICall.uv_index = response_json['currently']['uvIndex']
    darkSkyAPICall.visibility = response_json['currently']['visibility']

    weather = "Summary: {}".format(darkSkyAPICall.summary)
    
    #include print of weather to make sure all tests are working
    
    print(weather)

## Setting up widget for address user input

In [17]:
address = widgets.Text(
    value='Ex: 4120 14th St NW',
    placeholder='',
    description='Address: ',
    disabled=False
)

In [58]:
address

Text(value='850 Quincy St NW', description='Address: ', placeholder='')

In [59]:
address_input = address.value + ", Washington, DC"

## Setting up widget for date user input

In [60]:
date = widgets.DatePicker(
    description='Pick a Date',
    disabled=False
)

In [63]:
date

DatePicker(value=datetime.date(2019, 6, 13), description='Pick a Date')

In [64]:
date_input = date.value

## Setting up widget for time user input

In [65]:
time = widgets.Dropdown(
    options=['12:00', '12:30', '01:00', '01:30', '02:00', '02:30', '03:00',
'03:30', '04:00', '04:30','05:00', '05:30', '06:00', '06:30', '07:00', '07:30', 
'08:00', '08:30', '09:00', '09:30', '10:00', '10:30', '11:00', '11:30'],
    value='09:00',
    description='Depart Time:',
    disabled=False,
)

am_pm = widgets.Dropdown(
    options=['AM','PM'],
    value='AM',
    description='AM or PM? ',
    disabled=False,
)

In [66]:
HBox([time, am_pm])

## Cleaning up and reformatting date and time info

### Converting 12h time and am/pm input to 24h time

In [67]:
time_input = time.value + ':00' + am_pm.value

def convert24(time_input): 
    if time_input[-2:] == "AM" and time_input[:2] == "12": 
        return "00" + time_input[2:-2]    
    elif time_input[-2:] == "AM": 
        return time_input[:-2] 
    elif time_input[-2:] == "PM" and time_input[:2] == "12": 
        return time_input[:-2] 
    else: 
        return str(int(time_input[:2]) + 12) + time_input[2:8] 
  
time_24 = convert24(time_input)
time_24

'09:00:00'

### Combining date and time into one object

In [68]:
time_input_clean = datetime.datetime.strptime(time_24, '%H:%M:%S').time()
mydatetime = datetime.datetime.combine(date_input, time_input_clean)
date_time_input = mydatetime.isoformat()
date_time_input

'2019-06-13T09:00:00'

## Geocoder and DarkSkyAPI Calls

### Check our function inputs one more time

In [69]:
print("Address: " + address_input + "    Date/Time: " + date_time_input)

Address: 850 Quincy St NW, Washington, DC    Date/Time: 2019-06-13T09:00:00


### Call functions, return weather summary to make sure it worked.

In [70]:
addressLookup(address_input)
darkSkyAPICall(addressLookup.lat,addressLookup.lon,date_time_input)

Summary: Partly Cloudy


## Reformatting date/time for modeling input

### Days of week

In [71]:
wkday_0 = 0
wkday_1 = 0
wkday_2 = 0
wkday_3 = 0
wkday_4 = 0
wkday_5 = 0
wkday_6 = 0

In [72]:
print([wkday_0, wkday_1, wkday_2, wkday_3, wkday_4, wkday_5, wkday_6])

[0, 0, 0, 0, 0, 0, 0]


In [73]:
if mydatetime.weekday() == 0:
    wkday_0 = 1
    wkday_1 = 0
    wkday_2 = 0
    wkday_3 = 0
    wkday_4 = 0
    wkday_5 = 0
    wkday_6 = 0
elif mydatetime.weekday() == 1:
    wkday_0 = 0
    wkday_1 = 1
    wkday_2 = 0
    wkday_3 = 0
    wkday_4 = 0
    wkday_5 = 0
    wkday_6 = 0
elif mydatetime.weekday() == 2:
    wkday_0 = 0
    wkday_1 = 0
    wkday_2 = 1
    wkday_3 = 0
    wkday_4 = 0
    wkday_5 = 0
    wkday_6 = 0       
elif mydatetime.weekday() == 3:
    wkday_0 = 0
    wkday_1 = 0
    wkday_2 = 0
    wkday_3 = 1
    wkday_4 = 0
    wkday_5 = 0
    wkday_6 = 0
elif mydatetime.weekday() == 4:
    wkday_0 = 0
    wkday_1 = 0
    wkday_2 = 0
    wkday_3 = 0
    wkday_4 = 1
    wkday_5 = 0
    wkday_6 = 0
elif mydatetime.weekday() == 5:
    wkday_0 = 0
    wkday_1 = 0
    wkday_2 = 0
    wkday_3 = 0
    wkday_4 = 0
    wkday_5 = 1
    wkday_6 = 0
elif mydatetime.weekday() == 6:
    wkday_0 = 0
    wkday_1 = 0
    wkday_2 = 0
    wkday_3 = 0
    wkday_4 = 0
    wkday_5 = 0
    wkday_6 = 1

In [74]:
print([wkday_0, wkday_1, wkday_2, wkday_3, wkday_4, wkday_5, wkday_6])

[0, 0, 0, 1, 0, 0, 0]


### Time of Day

In [75]:
tod_evening = 0
tod_midnight = 0
tod_morning = 0
tod_night = 0
tod_afternoon = 0
tod_early_morning = 0

In [76]:
print([tod_evening, tod_midnight, tod_morning, tod_night, tod_afternoon, tod_early_morning])

[0, 0, 0, 0, 0, 0]


In [77]:
if 23 <= mydatetime.hour:
    tod_evening = 0
    tod_midnight = 1
    tod_morning = 0
    tod_night = 0
    tod_afternoon = 0
    tod_early_morning = 0
if 0 <= mydatetime.hour < 2:
    tod_evening = 0
    tod_midnight = 1
    tod_morning = 0
    tod_night = 0
    tod_afternoon = 0
    tod_early_morning = 0
elif 2 <= mydatetime.hour < 5:
    tod_evening = 0
    tod_midnight = 0
    tod_morning = 0
    tod_night = 1
    tod_afternoon = 0
    tod_early_morning = 0
elif 5 <= mydatetime.hour < 8:
    tod_evening = 0
    tod_midnight = 0
    tod_morning = 0
    tod_night = 0
    tod_afternoon = 0
    tod_early_morning = 1
elif 8 <= mydatetime.hour < 11:
    tod_evening = 0
    tod_midnight = 0
    tod_morning = 1
    tod_night = 0
    tod_afternoon = 0
    tod_early_morning = 0
elif 11 <= mydatetime.hour < 14:
    tod_evening = 0
    tod_midnight = 0
    tod_morning = 0
    tod_night = 0
    tod_afternoon = 1
    tod_early_morning = 0
elif 14 <= mydatetime.hour < 17:
    tod_evening = 0
    tod_midnight = 0
    tod_morning = 0
    tod_night = 0
    tod_afternoon = 1
    tod_early_morning = 0
elif 17 <= mydatetime.hour < 20:
    tod_evening = 1
    tod_midnight = 0
    tod_morning = 0
    tod_night = 0
    tod_afternoon = 0
    tod_early_morning = 0
elif 20 <= mydatetime.hour < 23:
    tod_evening = 0
    tod_midnight = 0
    tod_morning = 0
    tod_night = 1
    tod_afternoon = 0
    tod_early_morning = 0

In [78]:
print([tod_evening, tod_midnight, tod_morning, tod_night, tod_afternoon, tod_early_morning])

[0, 0, 1, 0, 0, 0]


## Modeling!

### Set up dataframe of info for model input

In [79]:
list_values = [addressLookup.tract + addressLookup.block_group, darkSkyAPICall.precip_intensity,
                   darkSkyAPICall.temp, darkSkyAPICall.uv_index, darkSkyAPICall.wind_speed,
                   tod_evening, tod_midnight, tod_morning, tod_night, tod_afternoon, tod_early_morning,
                   wkday_0, wkday_1, wkday_2, wkday_3, wkday_4, wkday_5, wkday_6]
list_columns = ['BlockGroup', 'percip_intensity','temperature',
                           'uv_index','wind_speed','tod_Evening','tod_Midnight',
                           'tod_Morning','tod_Night','tod_Afternoon','tod_Early Morning',
                           'wkday_0.0','wkday_1.0','wkday_2.0','wkday_3.0','wkday_4.0',
                           'wkday_5.0','wkday_6.0']


In [80]:
df = pd.DataFrame([list_values],columns=list_columns)

In [81]:
df

,BlockGroup,percip_intensity,temperature,uv_index,wind_speed,tod_Evening,tod_Midnight,tod_Morning,tod_Night,tod_Afternoon,tod_Early Morning,wkday_0.0,wkday_1.0,wkday_2.0,wkday_3.0,wkday_4.0,wkday_5.0,wkday_6.0
0,0025022,0,75.99,2,8.43,0,0,1,0,0,0,0,0,0,1,0,0,0


### Predict!

In [82]:
predicted = loaded_model.predict(df)

In [83]:
print(predicted)

[2]


In [93]:
df['BlockGroup']

0    0025022
Name: BlockGroup, dtype: object

In [109]:
if predicted >=4:
    print('Take a Lyft!')
else:
    bus_options = pd.merge(df['BlockGroup'], df_bus, how='left',
        left_on='BlockGroup', right_on='census_index')
    print(bus_options[['Stop_Name','Routes_Available']])
    


                                  Stop_Name  Routes_Available
0  GEORGIA AVE-PETWORTH STATION + BUS BAY B            70, 79
1          NEW HAMPSHIRE AVE NW + SPRING RD        63, 64, H8
2  GEORGIA AVE-PETWORTH STATION + BUS BAY C  60, 62, 62v1, 63
